<h4>Writing like Shakespeare</h4>

In [1]:
import io
import sys

import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import pad_sequences

2024-04-10 23:17:18.327424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 23:17:18.327606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 23:17:18.427592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 23:17:18.632274: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 23:17:21.070314: W tensorflow/compiler/tf2

In [2]:
def build_data(text, Tx=40, stride=3):
    # input data will be the tokens from the real poems
    X = []
    # labels will be the next character that should come
    Y = []
    for i in range(0, len(text)-Tx, stride):
        X.append(text[i : i + Tx])
        Y.append(text[i + Tx])
    return X, Y


def on_epoch_end(epoch, logs):
    None

def vectorization(X, Y, n_x, char_indices, Tx=40):
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=bool)
    y = np.zeros((m, n_x), dtype=bool)

    for i, sentence in enumerate(X):
        for j, char in enumerate(sentence):
            x[i, j, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1

    return x, y

def sample(preds, chars, temperature):
    preds = np.asarray(preds).astype('float64')

    preds = np.log(preds)/temperature
    expected_preds = np.exp(preds)
    preds = expected_preds / np.sum(expected_preds)

    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p=probas.ravel())

    return out



def generate_output(model, chars, indices_char, char_indices, Tx):
    generated = ''
    user_input = input("Input the first word of the poem you want me to write: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(user_input).lower()
    generated += user_input

    sys.stdout.write("\nPoem:\n")
    sys.stdout.write(user_input)
    for i in range(400):
        x_pred = np.zeros((1, Tx, len(chars)))
        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, chars, temperature=1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue


In [3]:
print('Loading text data...')
text = open("shakespeare.txt", "r").read().lower()
Tx = 40
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print("Creating training set...")
X, Y = build_data(text, Tx, stride=3)

print('Vectorizing training set...')
x, y = vectorization(X, Y, n_x=len(chars), char_indices=char_indices)
model = load_model('models/model_shakespeare_kiank_350_epoch.h5')

Loading text data...
Creating training set...
Vectorizing training set...


2024-04-10 23:17:25.676625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-10 23:17:26.008486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-10 23:17:26.008588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-10 23:17:26.013316: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-10 23:17:26.013418: I external/local_xla/xla/stream_executor

In [4]:
# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=256, epochs=1)

123/123 [==============================] - 117s 876ms/step - loss: 2.8374


In [5]:
generate_output(model, chars, indices_char, char_indices, Tx)
# watch -n 0.5 nvidia-smi


Poem:
Shubhame,
swear more strive in ampily far bear,
but ance pripb the wornd
to with laldelust, here store my mest exsusing,
and falled in my this hid everan my nead,
asgeos beauty to bedase,
hafle chilt-cummere are this sichly growed,
a butner dispaidains nath evashinged leas,
hake and where when thou bosf woest bewhy
so cange falled my faorer my wasted coinly?
 
whench thou secht o'pais that me didping to 